# Obtain data of AGN from SIMBAD, filtering the objects with data of [Ne II], [Ne III] and [Ne V] in infrared from NED Phot Tables and saving it

In this notebook we develop the code to Obtain the data of AGNs type Sy1 (Seyfert 1), Sy2 (Seyfert 2) and SyG (Seyfert Galaxies) from **[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-tap)** (SMB) and then search each object from the SMB sample in **[NED](https://ned.ipac.caltech.edu/)** to obtain the photometric data and filter all the objects from the sample that contain data of the emission lines of Ne II, Ne III and Ne V in infrared.

The part of the code to obtain the data from SIMBAD is based in Andres Ramos's work to obtain Seyfert Samples (https://github.com/aframosp/AGNView).

In [1]:
# If this notebook will be run in Google Colab, first at all the next code must be run. If the notebook will be run in Jupyter
#then "comment" or omit this part 

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install astroquery

In [2]:
from datetime import date
import time
import requests
import numpy as np

import pyvo
from astroquery.ipac.ned import Ned
from astroquery.simbad import Simbad

from pathlib import Path
from requests import Request, Session

import astropy.units as u
from astropy.table import hstack, QTable, Table
from astropy.coordinates import SkyCoord

In the next cell we define the paths and create the folders where we will save the data downloaded from the different Data Bases:

In [3]:
path_raw='../Datos/Raw/'
Path(path_raw).mkdir(parents=True, exist_ok=True)
path_phot_ned = '../Datos/Work/Phot_Tables_NED/' #In this path wi will save the photometry tables for each object
Path(path_phot_ned).mkdir(parents=True, exist_ok=True)

Using the package **[PyVO](https://pyvo.readthedocs.io/en/latest/#)** we can access to remote data and services like the **TAP Service of SIMBAD [SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-tap)** to find and retrieve the astronomical data that we need, in this case: Seyfert Galaxies. To achive that, we use a script with **[ADQL](https://www.ivoa.net/documents/latest/ADQL.html) (Astronomical Data Query Language)**, which is based on SQL.

On the script we used the next **Field Names** for VOTable ouput:
- ra: Right ascension
- dec: Declination
- coo_bibcode: Bibliographical reference for coordinates
- otype_txt: Object type
- rvz_bibcode: Bibliographical reference of radial velocity and redshift
- rvz_radvel: Stored value. Either a radial velocity, or a redshift, depending on the rvz_type field 
- rvz_redshift
- rvz_type: Stored type of velocity: 'v'=radial velocity, 'z'=redshift
- nbref: bibliographical reference

~~~~sql
SELECT main_id
    ,ra
    ,dec
    ,coo_bibcode
    ,otype_txt
    ,rvz_bibcode
    ,rvz_radvel
    ,rvz_redshift
    ,rvz_type
    ,nbref
    ,ids.ids
    ,alltypes.otypes
FROM basic
JOIN ids ON oid = ids.oidref
JOIN alltypes ON oid = alltypes.oidref
WHERE basic.otype = 'Sy1'
    OR basic.otype = 'Sy2'
    OR basic.otype = 'SyG'
~~~~

In [4]:
service = pyvo.dal.TAPService("http://simbad.u-strasbg.fr:80/simbad/sim-tap")

result = service.search("""
SELECT main_id
    ,ra
    ,dec
    ,coo_bibcode
    ,otype_txt
    ,rvz_bibcode
    ,rvz_radvel
    ,rvz_redshift
    ,rvz_type
    ,nbref
    ,ids.ids
    ,alltypes.otypes
FROM basic
JOIN ids ON oid = ids.oidref
JOIN alltypes ON oid = alltypes.oidref
WHERE basic.otype = 'Sy1'
    OR basic.otype = 'Sy2'
    OR basic.otype = 'SyG'
""")

We extract the object name (or in this case the main id) from the sample result.

In [5]:
simbad_sample_Sy = result.to_table()
obj_name_SMB=simbad_sample_Sy["main_id"]
obj_ids_SMB=simbad_sample_Sy["ids"]
len(simbad_sample_Sy)

44270

We save the date in a variable for use later in the file names of the results

In [6]:
today = date.today()
today.strftime('%b_%d_%Y')

'Sep_02_2022'

In [7]:
INIT_URL = 'http://vizier.u-strasbg.fr/viz-bin/sed?-c='
urls = [INIT_URL+str(simbad_sample_Sy['ra'][row])+'%20'+
        str(simbad_sample_Sy['dec'][row])+'&-c.rs=5' for row in range(len(simbad_sample_Sy))]
simbad_sample_Sy['cds_url'] = urls

Cell for search a specific id in the SMB sample

In [8]:
idS = ['3C 234.0'] #Id to search
idx = [np.where(simbad_sample_Sy['main_id'] == idt)[0] for idt in idS]
print(obj_name_SMB[idx[0]])
print('SMB sample index: ',idx[0][0])
obj_smb = obj_name_SMB[2789:2795]

main_id 
--------
3C 234.0
SMB sample index:  283


The sample results is saved in a .vot file:

In [9]:
simbad_sample_Sy.write(path_raw+'SMB_'+today.strftime('%b_%d_%Y')+'_Sy_Samples'+'.vot',
                    format='votable',overwrite=True)

### Function to download the Photometry tables from [NED](https://ned.ipac.caltech.edu/).

To get the photometry tables from NED, we use **[Astroquery](https://astroquery.readthedocs.io/en/latest/index.html)** package. In the function **photNED** first we try to obtain the photometry table with the ID name of the object from the SIMBAD sample. If there is not result, then with Astroquery we will search the others objects ID, and with that, search the information in NED. If finallly is not possible to find information with the objects ID, then we will try using its coordinates.

In [10]:
def photNED(name,oids,coord):
    
    '''
    This function is to get the photometry tables
    from NED. The input are the id and coordinates
    of the object.
    '''
    
    phot_ned = QTable()
    id_used = []
    
    try:
        phot_ned = Ned.get_table(name, table='photometry') #We get the Photometry Table using first SIMBAD names
        id_used = name
        #print('Using main SMB id\n')
    except:
        other_ids = oids.split('|')
        #other_ids = Simbad.query_objectids(name) #We get other IDs for the object from SIMBAD using astroquery
        for other_id in other_ids:
            try:
                phot_ned = Ned.get_table(other_id, table='photometry') #Photometry Table using other SIMBAD names
                id_used = other_id
                #print('Trying other id')
                break
            except:
                pass
    if id_used == []:
        try:
            region = Ned.query_region(coord,radius= 5 * u.arcsec)
            id_ned = region["Object Name"]
            phot_ned = Ned.get_table(id_ned, table='photometry')
            id_used = id_ned
            #print('Using region\n')
        except:
            pass

    return phot_ned, id_used

Using **SkyCoord** from astropy we extract the coordinates information for each object from the SIMBAD samples. This is useful if we want to search information in [NED](https://ned.ipac.caltech.edu/) using coordinates instead the object name

In [11]:
cat_smb_sample = SkyCoord(ra=simbad_sample_Sy['ra'], dec=simbad_sample_Sy['dec'])

### Filtering the Sample: Objects with [Ne II], [Ne III] and [Ne V] information. Downloading the NED Tables:

The wavelenght for NeII, NeIII and NeV in the infrared spectrum are 12.8 microns, 15.6 microns and 14.3 microns respectively, but in some cases inside NED tables there is also some filters for this lines reported in 12.81 microns for [Ne II], 15.5, 15.55 and 15.56 for [Ne III] and 14.32 for [Ne V].

* Wavelenght 12.8 microns -> Frequency 2.344e+13 Hz ([Ne II])
* Wavelenght 12.81 microns -> Frequency 2.342e+13 Hz ([Ne II])
* Wavelenght 14.3 microns -> Frequency 2.098e+13 Hz ([Ne V])
* Wavelenght 14.32 microns -> Frequency 2.095e+13 Hz ([Ne V])
* Wavelenght 15.5 microns -> Frequency 1.935e+13 Hz ([Ne III])
* Wavelenght 15.55 microns -> Frequency 1.929e+13 Hz ([Ne III])
* Wavelenght 15.56 microns -> Frequency 1.928e+13 Hz ([Ne III])
* Wavelenght 15.6 microns -> Frequency 1.923e+13 Hz ([Ne III])


Because in NED tables the frequency has a rounded value for three significant numbers, then we will use the next values: 

* Frequency: 1.92e+13 Hz: [NeIII]
* Frecuency: 1.93e+13 Hz: [Ne III]
* Frecuency: 2.09e+13 Hz: [Ne V]
* Frequency: 2.10e+13 Hz: [Ne V]
* Frequency: 2.34e+13 Hz: [Ne II

In the variable **Ne_IR_Fq** we define a list with the those frequency values

In [12]:
Ne_IR_Fq = np.array([19200000000000.0,19300000000000.0,20900000000000.0,21000000000000.0,23400000000000.0])*u.Hz

The filter for the objects with Ne Line Emissions in Infrared will be looking in the column **"Frequency"** of the photometry tables that we got from NED the values of the list defined in Ne_IR_Freq. Then, in a second fliter we are going to classify according to what information the Ne II, Ne III and Ne V emission lines each have.

In [13]:
def FilterbyNeFreq(NEDTable, Freq):
    
    NEDTable.remove_rows(np.where(NEDTable['Flux Density'].mask)[0])
    NEDTable.remove_rows(np.where(NEDTable['NED Units'] == 'Jy')[0])
    #NEDTable.remove_rows(np.where(NEDTable['NED Uncertainty'] == '')[0])
    #NEDTable.remove_rows(np.where(NEDTable['NED Uncertainty'] == '+/-...')[0])
    NEDTable.remove_rows(np.where(NEDTable['Observed Passband'] == '[Cl II] line (IRS)')[0])
    NEDTable.remove_rows(np.where(NEDTable['Observed Passband'] == '[Cl II] 14.4 (IRS)')[0])
    NEDTable.remove_rows(np.where(NEDTable['Observed Passband'] == '[Cl II] (IRS)')[0])
    NEDTable.remove_rows(np.where(NEDTable['Observed Passband'] == '[Cl II] 14.37 (IRS)')[0])
    
    f = np.array([NEDTable['Frequency']])*u.Hz #We get the Frequency column from the table
    intersec = np.intersect1d(Freq, f) #Intersection between Fr and Ne_IR_Fq
   
    return intersec    

In [ ]:
NeIR_obj_t = QTable(names=('Main_Id','Id_used_NED','RA','DEC','otype_txt','redshift',
                           'rvz_type','ids','otypes','cds_url','SMB_sample_index'),
                    dtype=('O','O','f8','f8','O','f8','U1','O','O','U93','f8'))

Ne_Inf = []

print(f"{'#':4} {'Idx':6} {'Name ID':^27} {'ID used in NED':^27} {'Ne IR Info':^20}")
i=-1
start = time.time()
for ind, id_smb in enumerate(obj_name_SMB):
    try:       
        #print('\nIdx SIMBAD: ',ind,' main_id: ',id_smb)
        phot_T,name_NED = photNED(id_smb,obj_ids_SMB[ind],cat_smb_sample[ind])
        #Save table again for work with it before save the original one
        phot_t,name_ned = photNED(id_smb,obj_ids_SMB[ind],cat_smb_sample[ind])
        ins = FilterbyNeFreq(phot_t, Ne_IR_Fq) #Search for the [Ne] frequencies in NED table
        
        if len(ins)>0: #First filter: If the phot table has infomation of frequencies of Fq            
            #print(ins)
            i=i+1
            phot_T.write(path_phot_ned+id_smb+'_NED_phot_tables'+'.vot', #Save the phot table in a vot table file
                            format='votable',overwrite=True)
            NeIR_obj_row = [id_smb,name_ned,simbad_sample_Sy["ra"][ind],simbad_sample_Sy["dec"][ind],
                            simbad_sample_Sy["otype_txt"][ind],simbad_sample_Sy["rvz_redshift"][ind],
                            simbad_sample_Sy["rvz_type"][ind],simbad_sample_Sy["ids"][ind],
                            simbad_sample_Sy["otypes"][ind],simbad_sample_Sy["cds_url"][ind],ind] 
            NeIR_obj_t.add_row(vals=NeIR_obj_row)
            
            if (Ne_IR_Fq[0] in ins or Ne_IR_Fq[1] in ins)and (Ne_IR_Fq[2] in ins or Ne_IR_Fq[3] in ins)\
            and Ne_IR_Fq[4] in ins: #Second filter to classify according the Ne info after remove empty flux values
                Ne_info = 'NeII|NeIII|NeV'
            elif (Ne_IR_Fq[0] in ins or Ne_IR_Fq[1] in ins) and Ne_IR_Fq[4] in ins:
                Ne_info = 'NeII|NeIII'
            elif (Ne_IR_Fq[2] in ins or Ne_IR_Fq[3] in ins) and Ne_IR_Fq[4] in ins:
                Ne_info = 'NeII|NeV'
            elif (Ne_IR_Fq[0] in ins or Ne_IR_Fq[1] in ins) and (Ne_IR_Fq[2] in ins or Ne_IR_Fq[3] in ins):
                Ne_info = 'NeIII|NeV'
            elif Ne_IR_Fq[0] in ins or Ne_IR_Fq[1] in ins:
                Ne_info = 'NeIII'
            elif Ne_IR_Fq[2] in ins or Ne_IR_Fq[3] in ins:
                Ne_info = 'NeV'
            elif Ne_IR_Fq[4] in ins:
                Ne_info = 'NeII'
            else:
                Ne_info = 'No Flux Values' 
            Ne_Inf.append(Ne_info)
            print(f"{'%g'%i:4} {'%g'%ind:6} {'%s'%id_smb:^27} {'%s'%name_ned:^27} {'%s'%Ne_info:^20}")
    
    except KeyError:
        pass

end = time.time()

print('\nTotal objects with lines emission of NeII or NeIII or NeV in IR  found: ',len(NeIR_obj_t))
print('Execution Time(seg): ' + str(end - start))
print('Execution Time(min): ' + str((end - start)/60))
print('Execution Time(hrs): ' + str((end - start)/3600))

#    Idx              Name ID                 ID used in NED             Ne IR Info     
0    283             3C 234.0                    3C 234.0              NeII|NeIII|NeV   
1    428      2MASS J13000535+1632148       NVSS J130005+163212          NeII|NeIII     
2    484             NGC   262                   NGC   262             NeII|NeIII|NeV   
3    541          IRAS 04385-0828             IRAS 04385-0828          NeII|NeIII|NeV   
4    628             NGC  4945                   NGC  4945             NeII|NeIII|NeV   
5    654              IC 4553                     IC 4553                NeII|NeIII     
6    852             Mrk  279                    Mrk  279              NeII|NeIII|NeV   
7    1227            UGC 11680                   UGC 11680                  NeII        
8    1246          LEDA   45656                LEDA   45656               NeII|NeV      
9    1282            Mrk 1239                    Mrk 1239              NeII|NeIII|NeV   
10   1341          LE

93   14382           NGC  5728                   NGC  5728             NeII|NeIII|NeV   
94   14458          ESO 140-43                  ESO 140-43             NeII|NeIII|NeV   
95   14562            IC 1816                     IC 1816              NeII|NeIII|NeV   
96   14590   SDSS J161647.32+371621.2    SDSS J161647.32+371621.2        NeIII|NeV      
97   14665           ESO 417-6                   ESO 417-6             NeII|NeIII|NeV   
98   14861           NGC   788                   NGC   788             NeII|NeIII|NeV   
99   14910            2C 1066                   1Jy 1254+47              NeII|NeIII     
100  14946            IC 5298                     IC 5298                   NeV         
101  14957            IC 5063                     IC 5063              NeII|NeIII|NeV   
102  15090           PB  3894                    PB  3894                NeIII|NeV      
103  15224             M  88                       M  88                 NeII|NeIII     
104  15262           

In [ ]:
NeIR_obj_t['NED_Ne_IR_info'] = Ne_Inf #Add a column to save the Ne info
NeIR_obj_t.show_in_notebook()

In [ ]:
NeIR_obj_t.write(path_raw+'Obj_Ne-IR_'+today.strftime('%b_%d_%Y')+'.vot',
                    format='votable',overwrite=True)

In [ ]:
NeII_NeIII_NeV_IR = np.where(NeIR_obj_t['NED_Ne_IR_info'] == 'NeII|NeIII|NeV')[0] #List of objects with NeII, NeIII and NeV data
NeII_NeIII_IR = np.where(NeIR_obj_t['NED_Ne_IR_info'] == 'NeII|NeIII')[0] #List of objects with NeII and NeIII data
NeII_NeV_IR = np.where(NeIR_obj_t['NED_Ne_IR_info'] == 'NeII|NeV')[0] #List of objects with NeII and NeV data
NeIII_NeV_IR = np.where(NeIR_obj_t['NED_Ne_IR_info'] == 'NeIII|NeV')[0] #List of objects with NeIII and NeV data
NeIII_IR = np.where(NeIR_obj_t['NED_Ne_IR_info'] == 'NeIII')[0] #List of objects with NeIII data
NeV_IR = np.where(NeIR_obj_t['NED_Ne_IR_info'] == 'NeV')[0] #List of objects with NeV data
NeII_IR = np.where(NeIR_obj_t['NED_Ne_IR_info'] == 'NeII')[0] #List of objects with NeII data
No_Ne_IR = np.where(NeIR_obj_t['NED_Ne_IR_info'] == 'No Flux Values')[0] #List of objects with NeII data
print('Total objects found with NeII, NeIII and NeV emission lines: ',len(NeII_NeIII_NeV_IR))
print('Total objects found with NeII and NeIII emission lines: ',len(NeII_NeIII_IR))
print('Total objects found with NeII and NeV emission lines: ',len(NeII_NeV_IR))
print('Total objects found with NeIII and NeV emission lines: ',len(NeIII_NeV_IR))
print('Total objects found with NeIII emission lines: ',len(NeIII_IR))
print('Total objects found with NeV emission lines: ',len(NeV_IR))
print('Total objects found with NeII emission lines: ',len(NeII_IR))
print('Total objects found with No Ne flux values: ',len(No_Ne_IR))

## Notebook Info

In [ ]:
%reload_ext watermark
%watermark -a "Jonhatan Bernal" -d -v -m
print('Specific Python packages')
%watermark -iv -w --packages astropy

### Las celdas a continuación son para pruebas:

In [ ]:
x=283
simbad_sample_Sy[x]

In [ ]:
nam=simbad_sample_Sy['main_id'][x]
other_ids = Simbad.query_objectids(nam)
other_ids

In [ ]:
for _id in other_ids['ID']:
    try:
        ned_tab= Ned.get_table(_id, table='photometry')
        print('si ',_id)
        break   
    except:
        print('no ',_id)
        if _id == other_ids['ID'][-1]:
            continue

In [ ]:
obj_ids_SMB[x]

In [ ]:
o_ids = obj_ids_SMB[x].split('|')
for other_id in o_ids:
    print(other_id)